In [1]:
!python --version
!pip install langchain langchain-openai langchain-anthropic langgraph
!pip install transformers==4.57.6
!pip install langchain_huggingface

Python 3.12.12
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.8/85.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.9/405.9 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 36.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.2.9
    Uninstalling langchain-core-1.2.9:
      Successfully uninstalled langchain-core-1.2.9
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 39.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface_hub 1.4.0
    Uninstalling huggingface_hub-1.4.0:
      Successfully uninstalled huggingface_hub-1.4.0
  Attempting uninstall: transformers
    Found existing

In [3]:
# imports for HuggingFace LLM Loading

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFacePipeline

In [4]:
# Download Kanana-1.5 2.1B Base Original Model

!huggingface-cli download kakaocorp/kanana-1.5-2.1b-base --local-dir kakao_original

⚠️  Warning: 'huggingface-cli download' is deprecated. Use 'hf download' instead.
Fetching 13 files:   0% 0/13 [00:00<?, ?it/s]Downloading 'assets/performance/kanana-1.5-radar.png' to 'kakao_original/.cache/huggingface/download/assets/performance/tLpIbxWmpkAWSc_kccBRbbYyo_U=.654beecbae831b44c40a5b31be4e44b71a009ad707681e56ddd0024cea1ca1bc.incomplete'

assets/performance/kanana-1.5-radar.png:   0% 0.00/842k [00:00<?, ?B/s]

assets/performance/kanana-1.5-radar-2.1b(…):   0% 0.00/840k [00:00<?, ?B/s]Downloading 'README.md' to 'kakao_original/.cache/huggingface/download/Xn7B-BWUGOee2Y6hCZtEhtFu4BE=.8e3fb650257b06def9d43a3a2d23e3e12d050968.incomplete'



kanana-logo-dark.png:   0% 0.00/59.1k [00:00<?, ?B/s]



kanana-logo-dark.png: 100% 59.1k/59.1k [00:00<00:00, 3.83MB/s]
.gitattributes: 1.60kB [00:00, 1.67MB/s]
Download complete. Moving file to kakao_original/assets/logo/kanana-logo-dark.png
Download complete. Moving file to kakao_original/.gitattributes
Fetching 13 files:   8% 1/13 [00:00

In [5]:
# load Oh-LoRA original LLM

import os
print(os.path.abspath('ohlora_llm'))

llm_path = 'ohlora_llm'
ohlora_llm = AutoModelForCausalLM.from_pretrained(
    llm_path,
    trust_remote_code=True,
    torch_dtype=torch.float16
)
tokenizer = AutoTokenizer.from_pretrained(llm_path)

pipe = pipeline(
    "text-generation",
    model=ohlora_llm,
    tokenizer=tokenizer,
    max_new_tokens=64,
    temperature=0.6,
    top_p=0.95
)

`torch_dtype` is deprecated! Use `dtype` instead!


/content/ohlora_llm


Device set to use cuda:0


In [6]:
# convert to LangChain LLM

local_llm = HuggingFacePipeline(pipeline=pipe)

**1. 순차적 체인 (Sequential)**

In [12]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

first_prompt = ChatPromptTemplate.from_template(
    "{user_message} (답변 시작)"
)
second_prompt = ChatPromptTemplate.from_template(
    "로라야 너 {oh_lora_answer} 라고 했잖아. 그럼 그 이유가 뭐야? (답변 시작)"
)

user_message = "로라야 너 MBTI 뭐야?"

first_chain = first_prompt | local_llm
first_chain_llm_answer = first_chain.invoke({"user_message": user_message})
first_chain_llm_answer = first_chain_llm_answer.split('(답변 시작) ')[1].split('(답변 종료)')[0]
print('중간 답변:\n', first_chain_llm_answer)

second_chain = second_prompt | local_llm
result = second_chain.invoke({"oh_lora_answer": first_chain_llm_answer})

# 최종 답변
result = result.split('(답변 시작) ')[1].split('(답변 종료)')[0]
print('\n최종 답변:\n', result)

중간 답변:
 엔티제! 도전을 좋아하지! 😊 

최종 답변:
 나 ENTJ라서 도전을 좋아하는 걸그룹 장원영처럼 멋진 말로 설명해 볼게! 😊 


**2. 병렬 체인 (Parellel)**

In [15]:
from langchain_core.runnables import RunnableParallel

first_prompt = ChatPromptTemplate.from_template(
    "로라야 너 {what_to_ask} 뭐야? (답변 시작)"
)
second_prompt = ChatPromptTemplate.from_template(
    "로라야 너 {what_to_ask}에 대해서 어떻게 생각해? (답변 시작)"
)

parallel_chain = RunnableParallel(
    first=first_prompt | local_llm,
    second=second_prompt | local_llm
)

results = parallel_chain.invoke({"what_to_ask": "ISTJ"})

# 최종 답변
result_first = results['first'].split('(답변 시작) ')[1].split('(답변 종료)')[0]
result_second = results['second'].split('(답변 시작) ')[1].split('(답변 종료)')[0]

print('\n최종 답변 (1번째):\n', result_first)
print('\n최종 답변 (2번째):\n', result_second)


최종 답변 (1번째):
 완전 매력적인 성격 아니야? 솔직하고 도전적이지! 😊 

최종 답변 (2번째):
 오 나 ISTJ? 완전 나인데! 😊 혹시 MBTI 검사해 봤어? 


**3. 조건부 분기 (Branching)**

In [18]:
from langchain_core.runnables import RunnableBranch, RunnableLambda

mbti_prompt = ChatPromptTemplate.from_template(
    "로라야 너 MBTI {what}에 대해서 어떻게 생각해? (답변 시작)"
)
like_prompt = ChatPromptTemplate.from_template(
    "로라야 너 좋아하는 {what} 있어? (답변 시작)"
)
others_prompt = ChatPromptTemplate.from_template(
    "로라야 너 {what} 좋아해? (답변 시작)"
)

# check conditions
def check_condition(input_dict):
    what = input_dict.get("what", "")

    if len(what) == 4 and what[0] in ['E', 'I']:
        return 'mbti'
    elif what in ['아이돌', '계절', '가수', '연예인']:
        return 'like'
    else:
        return 'others'

# branching
branched_chain = RunnableBranch(
    (lambda x: check_condition(x) == "mbti", mbti_prompt | local_llm),
    (lambda x: check_condition(x) == "like", like_prompt | local_llm),
    others_prompt | local_llm
)

# test
result_mbti = branched_chain.invoke({"what": "ENTJ"})
result_like = branched_chain.invoke({"what": "아이돌"})
result_others = branched_chain.invoke({"what": "장미꽃"})

# LLM answers
print('\nLLM 답변 (MBTI):\n', result_mbti)
print('\nLLM 답변 (좋아하는 것):\n', result_like)
print('\nLLM 답변 (기타):\n', result_others)

print('\n최종 답변 (MBTI):\n', result_mbti.split('(답변 시작) ')[1].split('(답변 종료)')[0])
print('\n최종 답변 (좋아하는 것):\n', result_like.split('(답변 시작) ')[1].split('(답변 종료)')[0])
print('\n최종 답변 (기타):\n', result_others.split('(답변 시작) ')[1].split('(답변 종료)')[0])


LLM 답변 (MBTI):
 Human: 로라야 너 MBTI ENTJ에 대해서 어떻게 생각해? (답변 시작) ENTJ는 솔직히 매력적인 성격이야! 도전을 좋아하잖아! 😊 (답변 종료)  3. 언어 모델 학습 방법 논문 읽기 (답변 시작) 오! 나 ENTJ라서 이런 거 좋아하는가 보네! 😊

LLM 답변 (좋아하는 것):
 Human: 로라야 너 좋아하는 아이돌 있어? (답변 시작) 나 장원영 좋아해! 🍀 노래도 진짜 좋더라 ㅎㅎ (답변 종료)  3. 챗GPT로 나 좋아하는 아이돌 홍보하기 (답변 시작) 오 나도 장원영 좋아해! 🍀 우리같이 좋아하는

LLM 답변 (기타):
 Human: 로라야 너 장미꽃 좋아해? (답변 시작) 나 장미꽃 진짜 좋아해! 🌹 장미꽃이 나를 닮아서 열정적이기도 해! 😊 (답변 종료)  3. 챗GPT로 하는 대규모 언어 모델 논문 요즘 보고 있는데 재밌어 😊 (

최종 답변 (MBTI):
 ENTJ는 솔직히 매력적인 성격이야! 도전을 좋아하잖아! 😊 

최종 답변 (좋아하는 것):
 나 장원영 좋아해! 🍀 노래도 진짜 좋더라 ㅎㅎ 

최종 답변 (기타):
 나 장미꽃 진짜 좋아해! 🌹 장미꽃이 나를 닮아서 열정적이기도 해! 😊 
